In [1]:
import pandas as pd
from shapely.wkt import loads
import geopandas as gpd
from shapely.geometry import Point, Polygon

import openrouteservice as ors

from loguru import logger

from tqdm import tqdm

import json

from datetime import date, datetime

from ast import literal_eval

import itertools


In [2]:
network = 'amm_network'
path = "G:\Mi unidad\walknet_datalake"

<>:2: SyntaxWarning: invalid escape sequence '\M'
<>:2: SyntaxWarning: invalid escape sequence '\M'
C:\Users\katul\AppData\Local\Temp\ipykernel_2756\3793340662.py:2: SyntaxWarning: invalid escape sequence '\M'
  path = "G:\Mi unidad\walknet_datalake"


In [3]:
def storage_file(path,network):
    # Open json file and read into dict
    with open(r"{path}\sources\ors\level0\level0_{network}.json".format(path = path, network=network),) as f:
        data = json.load(f)
        data = {datetime.strptime(date, '%d/%m/%Y').date():{literal_eval(c): loads(p) for c,p in calls.items()} for date, calls in data.items()}
        return data

In [4]:
def make_measure_points(network, path, round_to = 2):
    nodes = pd.read_csv(
        r"{path}\networks\{network}\nodes.csv".format(path = path, network=network),
        sep=";")
    nodes['geometry'] = nodes['geometry'].apply(lambda x: loads(x))
    nodes = gpd.GeoDataFrame(nodes, geometry = 'geometry', crs=25830)
    nodes = nodes.to_crs(4326)

    round_nodes = gpd.GeoDataFrame(
        nodes,
        geometry = nodes.apply(lambda x: Point((round(x['geometry'].x,round_to),round(x['geometry'].y,round_to))),axis=1),
        crs = 4326)
    
    coords = tuple(round_nodes.apply(lambda row: (row['geometry'].x , row['geometry'].y),axis=1).unique())

    return coords

def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

In [5]:
def get_isochrones(data, coords):
    
    client = ors.Client(key='5b3ce3597851110001cf6248e310e50239484390adc4935f51817f90')
    processed_coords = tuple(set(list(itertools.chain.from_iterable([list(v.keys()) for v in data.values()]))))
    
    if date.today() in data.keys():
        limit = 500 - len(data[date.today()])
    else: 
        limit = 500
        data[date.today()] = {}

    coords = tuple(c for c in coords if not c in processed_coords)[0:limit]

    for chunk in tqdm(chunker(coords,5)):
        
        isochrone = ors.isochrones.isochrones(
            client,
            chunk,
            profile='driving-car',
            range_type='time',
            range=[30],
            units=None,
            location_type=None,
            smoothing=0.2,
            attributes=None,
            validate=True,
            dry_run=None)['features']

        for i,feature in enumerate(isochrone):
            #print(f"Processing feature {feature}")
            
            if feature['geometry']['type'] == 'Polygon':
                data[date.today()].update({chunk[i] : Polygon(feature['geometry']['coordinates'][0])})
    return data

In [ ]:
def dump_data(path, network, data):
    processed_data = json.dumps({date.strftime('%d/%m/%Y'):{f"({k[0]},{k[1]})":v.wkt for k,v in iso.items()} for date, iso in data.items()})
    with open(r"{path}\sources\ors\level0\level0_{network}.json".format(path = path, network=network), 'w') as f:
        json.dump(processed_data, f)

In [6]:
processed = storage_file(path, network)
coords = make_measure_points(network, path, round_to = 2)
today_processed = get_isochrones(processed, coords)
dump_data(path, network, today_processed)



19it [00:02, 13.61it/s]c:\Documents\Github\pw_sources\env\Lib\site-packages\openrouteservice\client.py:211: UserWarning: Rate limit exceeded. Retrying for the 1st time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
c:\Documents\Github\pw_sources\env\Lib\site-packages\openrouteservice\client.py:211: UserWarning: Rate limit exceeded. Retrying for the 2nd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
c:\Documents\Github\pw_sources\env\Lib\site-packages\openrouteservice\client.py:211: UserWarning: Rate limit exceeded. Retrying for the 3rd time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
c:\Documents\Github\pw_sources\env\Lib\site-packages\openrouteservice\client.py:211: UserWarning: Rate limit exceeded. Retrying for the 4th time.
  warnings.warn('Rate limit exceeded. Retrying for the {0}{1} time.'.format(retry_counter + 1,
c:\Documents\